In [43]:
%run Flightprice_preparation.ipynb

In [50]:
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd

In [51]:
df

,Unnamed: 0,Unnamed: 0.1,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,0,0,9ca0e81111c683bec1012473feefd28f,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H29M,0,...,1650223560,2022-04-17T15:26:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,8940,947,coach
1,1,1,98685953630e772a098941b71906592b,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0,...,1650200400,2022-04-17T09:00:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9000,947,coach
2,2,2,98d90cbc32bfbb05c2fc32897c7c1087,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0,...,1650218700,2022-04-17T14:05:00.000-04:00,BOS,ATL,Delta,DL,Boeing 757-200,9000,947,coach
3,3,3,969a269d38eae583f455486fa90877b4,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H32M,0,...,1650227460,2022-04-17T16:31:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9120,947,coach
4,4,4,980370cf27c89b40d2833a1d5afc9751,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H34M,0,...,1650213180,2022-04-17T12:33:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9240,947,coach
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,9995,4a3eefb4e0fc5ae3ebe67b60c658e7c2,2022-04-16,2022-04-18,DEN,BOS,VAA0PHEN,PT7H33M,0,...,1650300600||1650314640,2022-04-18T12:50:00.000-04:00||2022-04-18T16:4...,EWR||BOS,DEN||EWR,United||United,UA||UA,Boeing 737-900||Airbus A319,13140||4440,1621||185,coach||coach
9996,9996,9996,5ffee9f5095d952ddc84243c9851d702,2022-04-16,2022-04-18,DEN,BOS,KA0NA0MC,PT7H36M,0,...,1650295140||1650312660,2022-04-18T11:19:00.000-04:00||2022-04-18T16:1...,DTW||BOS,DEN||DTW,Delta||Delta,DL||DL,Airbus A321||Airbus A321,9840||6960,1142||622,coach||coach
9997,9997,9997,f85e3490e7e0293e2886f62bc78b365f,2022-04-16,2022-04-18,DEN,BOS,KA0NA0MC,PT7H39M,0,...,1650318960||1650339840,2022-04-18T16:56:00.000-05:00||2022-04-18T23:4...,MSP||BOS,DEN||MSP,Delta||Delta,DL||DL,Airbus A321||Boeing 737-900,6660||9960,693||1122,coach||coach
9998,9998,9998,5cfdca4b711239a64fe84ed5fa03a30e,2022-04-16,2022-04-18,DEN,BOS,KA0NA0MC,PT7H41M,0,...,1650310500||1650324360,2022-04-18T15:35:00.000-04:00||2022-04-18T19:2...,LGA||BOS,DEN||LGA,Delta||Delta,DL||DL,Boeing 737-800||Embraer 175,13800||5160,1621||185,coach||coach


In [52]:
# Prepared data for training 
X = df_processed.drop("totalFare", axis =1).copy()
y = df_processed[["totalFare"]]


Xtrain, Xtest, Xdev, ytrain, ytest, ydev = split_train_test_dev(X, y, flight_price_col='totalFare', test_ratio=0.2, dev_ratio=0.1)
Xtrain_prepared, Xtest_prepared, Xdev_prepared, ytrain_prepared, ytest_prepared, ydev_prepared = prepare_for_train(Xtrain, Xdev, Xtest, ytrain, ydev, ytest)


/opt/anaconda3/envs/ds420/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [53]:
# Algorithms to train 

models = {
    "SGDRegressor": SGDRegressor(max_iter=1000, tol=1e-3, random_state=42),
    "LinearRegression": LinearRegression(),
    "KNeighborsRegressor": KNeighborsRegressor(),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=42)
}

# Prepare a dataframe to store the RMSE and MAE
scores_df = pd.DataFrame(columns=["RMSE", "MAE"], index=models.keys())

# Train each model and compute scores
for model_name, model in models.items():
    model.fit(Xtrain_prepared, ytrain_prepared)  # Make sure to use your actual training data variable names
    y_pred = model.predict(Xdev_prepared)  # Evaluate on the dev set; adjust variable names as needed

    rmse = np.sqrt(mean_squared_error(ydev_prepared, y_pred))
    
    mae = mean_absolute_error(ydev_prepared, y_pred)

    scores_df.loc[model_name] = [rmse, mae]

scores_df



/opt/anaconda3/envs/ds420/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,RMSE,MAE
SGDRegressor,92.409709,33.983413
LinearRegression,11295813625.965757,3560533347.039368
KNeighborsRegressor,164.619695,61.505246
DecisionTreeRegressor,142.625712,21.856781


In [ ]:
# According to the performance metrics, SGDRegressor and DecisionTreeRegressor has the best 
# performance with low root mean squared error and mean absolute error. Therefore, we will use 
# these two algorithm in the next step 